In [1]:
import json
import numpy as np
import pandas as pd
import scipy.stats as st

import matplotlib.pyplot as plt

In [43]:
# Here we can add all the methods we have/want
models_list = ['clip_baseline',
    'coop_baseline', 'coop_pseudo_baseline',
    'vpt_baseline', 'vpt_pseudo_baseline', 
    'teacher_student',
]

In [95]:
model = models_list[1]
print(model)
filename = f"results/results_model_{model}.json"

data = [json.loads(i) for i in open(filename,'r').readlines()]

coop_baseline


In [97]:
std_accuracies = []
gen_accuracies = []
seen_accuracies = []
unseen_accuracies = []
harmonic_accuracies = []

for m in data:
    # Here change and filter data depending on what results we want/
    # For instance, let's assume we want 
    if (
        (m['config']['DATASET_NAME'] == 'DTD')  \
         and (m['config']['VIS_ENCODER'] == 'ViT-L/14') \
         and (m['config']['SPLIT_SEED'] == 500) \
    ):
        std_accuracies.append(m['std_accuracy'])
        gen_accuracies.append(m['gen_accuracy'])
        seen_accuracies.append(m['gen_seen'])
        unseen_accuracies.append(m['gen_unseen'])
        harmonic_accuracies.append(st.hmean([m['gen_seen'], m['gen_unseen']]))

print(f"Mean STD accuracy: {round((np.sum(std_accuracies)/len(std_accuracies))*100, 2)}")
print(f"Std STD accuracy: {np.std(std_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(std_accuracies)-1, loc=np.mean(std_accuracies), scale=st.sem(std_accuracies)) 
print(f"95% STD confidence interval {round((np.sum(std_accuracies)/len(std_accuracies) - interval[0])*100, 2)}")
print('\n')

print(f"Mean SEEN accuracy: {round((np.sum(seen_accuracies)/len(seen_accuracies))*100, 2)}")
print(f"Std SEEN accuracy: {np.std(seen_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(seen_accuracies)-1, loc=np.mean(seen_accuracies), scale=st.sem(seen_accuracies)) 
print(f"95% SEEN confidence interval {round((np.sum(seen_accuracies)/len(seen_accuracies) - interval[0])*100, 2)}")
print('\n')

print(f"Mean UNSEEN accuracy: {round((np.sum(unseen_accuracies)/len(unseen_accuracies))*100,2)}")
print(f"Std UNSEEN accuracy: {np.std(unseen_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(unseen_accuracies)-1, loc=np.mean(unseen_accuracies), scale=st.sem(unseen_accuracies)) 
print(f"95% UNSEEN confidence interval {round((np.sum(unseen_accuracies)/len(unseen_accuracies) - interval[0])*100,2)}")
print('\n')

print(f"Mean HARMONIC accuracy: {round((np.sum(harmonic_accuracies)/len(harmonic_accuracies))*100, 2)}")
print(f"Std HARMONIC accuracy: {np.std(harmonic_accuracies)}")
interval = st.t.interval(alpha=0.95, df=len(harmonic_accuracies)-1, loc=np.mean(harmonic_accuracies), scale=st.sem(harmonic_accuracies)) 
print(f"95% HARMONIC confidence interval {round((np.sum(harmonic_accuracies)/len(harmonic_accuracies) - interval[0])*100, 2)}")

Mean STD accuracy: 54.69
Std STD accuracy: 0.02189692463890707
95% STD confidence interval 3.04


Mean SEEN accuracy: 76.09
Std SEEN accuracy: 0.009670554249917359
95% SEEN confidence interval 1.34


Mean UNSEEN accuracy: 40.11
Std UNSEEN accuracy: 0.024158683227954326
95% UNSEEN confidence interval 3.35


Mean HARMONIC accuracy: 52.48
Std HARMONIC accuracy: 0.019526022951302967
95% HARMONIC confidence interval 2.71


In [29]:
unseen_accuracies

[0.3486111111111111,
 0.3486111111111111,
 0.3680555555555556,
 0.3527777777777778,
 0.35555555555555557,
 0.3638888888888889,
 0.3625,
 0.34305555555555556,
 0.3625,
 0.35,
 0.35138888888888886]

In the google sheets, I report the mean and the confidence interval.

In [18]:
import pickle

In [20]:
with open('Flowers102_coop_baseline_RN101_opt_100_spl_0.pickle', 'rb') as f:
    data = pickle.load(f)

In [23]:
data[0]

array([[[ 0.0141661 ,  0.01154805, -0.00731809, ...,  0.0088033 ,
          0.00533164,  0.00627134],
        [ 0.00435409,  0.0012099 , -0.00862264, ..., -0.00905044,
          0.01261343,  0.00636003],
        [ 0.00259562,  0.00175074, -0.00354808, ..., -0.01755268,
          0.00199584, -0.01359151],
        ...,
        [ 0.0136095 , -0.01302855, -0.00152316, ..., -0.00584143,
          0.01086486,  0.01312908],
        [-0.0004344 ,  0.0026134 , -0.00459469, ...,  0.0004229 ,
          0.01705721,  0.02296569],
        [ 0.0063952 ,  0.002413  , -0.00985158, ..., -0.00904753,
         -0.00120628, -0.00495799]]], dtype=float32)